In [28]:
import pandas as pd

import os
import zipfile

In [43]:
# load file alokasi petugas
dt_alokasi = pd.read_excel('petugas.xlsx',dtype=str)

In [44]:
#config file
tgl = '20260223'
folder_input = 'input_sosial'
folder_output = 'output'
file_anomali_zip_prov = '01. Anomali Provinsi-20260223T032415Z-1-001.zip'
file_anomali_zip_pusat = '02. Anomali Pusat-20260223T032517Z-1-001.zip'
file_anomali_zip_konsistensi = '03. Konsistensi-20260223T032535Z-1-001.zip'
file_kode_pusat = 'Daftar Anomali & Metadata Sakernas Februari 2026.xlsx'


In [45]:
#gabung anomali provinsi

# Nama file zip
zip_path = f'{folder_input}/{file_anomali_zip_prov}'
dt_anom_all = []

os.makedirs(f'{folder_output}/{tgl}', exist_ok=True)
# Buka zip
with zipfile.ZipFile(zip_path, 'r') as z:
    # Lihat daftar file di dalam zip
    # print(z.namelist())

    # Baca salah satu file (misalnya 'data.txt')
    for file in z.namelist():
        try:
            kode_anomali=file.split('_')[1]
            nama_anomali=" ".join(file.split('_')[2:-1])
            with z.open(file) as f:
                dt_anom = pd.read_excel(f,dtype='str')
                dt_anom['kode_anomali']=kode_anomali
                dt_anom['nama_anomali']=nama_anomali
                dt_anom=pd.merge(dt_anom,dt_alokasi,how='left',left_on=['KODE_WILAYAH','DSRT'],right_on=['KODE_WILAYAH','NUS'])
                keep_cols = [
                    'KODE_KAB',
                    'KODE_WILAYAH',
                    'kab_name',
                    'kec_name',
                    'desa_name',
                    'sls_name',
                    'DSRT','KRT','kode_anomali','nama_anomali','catatan','alamat','NO_BANG','longitude','latitude','email','petugas','role','LINK']
                # kolom yang mau digabung (exclude keep_cols)
                exclude_cols = [c for c in dt_anom.columns if c not in keep_cols]

                # buat kolom baru "extra_info"
                dt_anom['extra_info'] = dt_anom[exclude_cols].apply(
                lambda row: ";\n".join([f"{col} : {row[col]}" for col in exclude_cols if pd.notna(row[col])]),
                    axis=1
                )
                # hasil dataframe hanya berisi kolom utama + extra_info
                dt_result = dt_anom[keep_cols + ['extra_info']]
                dt_result = dt_result.sort_values(by=['KODE_KAB','KODE_WILAYAH' ,'DSRT','kode_anomali'])
                unik_kab = dt_result['KODE_KAB'].unique()

                for kab in unik_kab:
                    kab = kab.zfill(2)
                    dt_filter = dt_result[dt_result.KODE_KAB==kab]
                    os.makedirs(f'{folder_output}/{tgl}/{kab}', exist_ok=True)
                    dt_filter.to_excel(f"{folder_output}/{tgl}/{kab}/{kode_anomali}.xlsx")
                dt_anom_all.append(dt_result)
        except Exception as e:
            print(e)
            print(file)

In [46]:
#kode pusat
kode_pusat = pd.read_excel(file_kode_pusat,dtype=str,sheet_name='Daftar Anomali')

In [ ]:
# Nama file zip
zip_path = f'{folder_input}/{file_anomali_zip_pusat}'
#dt_anom_all = []
# Buka zip
with zipfile.ZipFile(zip_path, 'r') as z:
    # Lihat daftar file di dalam zip
    #print(z.namelist())
    for file in z.namelist():
        try:
            kode_anomali = 'A'+file.split('_')[5]
            nama_anomali=kode_pusat[kode_pusat['Kode Anomali']==kode_anomali].iloc[0]['keterangan']
            with z.open(file) as f:
                dt_anom = pd.read_excel(f,dtype='str')
                dt_anom['KODE_WILAYAH']=dt_anom['KODE_PROV']+dt_anom['KODE_KAB']+dt_anom['KODE_KEC']+dt_anom['KODE_DESA']+dt_anom['SLS']+dt_anom['SUBSLS']
                dt_anom['kode_anomali']=kode_anomali
                dt_anom['nama_anomali']=nama_anomali
                dt_anom=pd.merge(dt_anom,dt_alokasi,how='left',left_on=['KODE_WILAYAH','DSRT'],right_on=['KODE_WILAYAH','NUS'])
                #cek catatan
                if 'catatan' not in dt_anom.columns:
                    dt_anom['catatan']=''    
                #cek link
                if 'link' in dt_anom.columns:
                    dt_anom=dt_anom.rename(columns={'link' : 'LINK'})
                if 'LINK' not in dt_anom.columns:
                    dt_anom['LINK']=''
                keep_cols = ['KODE_KAB','KODE_WILAYAH',
                    'kab_name',
                    'kec_name',
                    'desa_name',
                    'sls_name',
                             'DSRT','KRT','kode_anomali','nama_anomali','catatan','alamat','NO_BANG','longitude','latitude','email','petugas','role','LINK']
                # kolom yang mau digabung (exclude keep_cols)
                exclude_cols = [c for c in dt_anom.columns if c not in keep_cols]

                # buat kolom baru "extra_info"
                dt_anom['extra_info'] = dt_anom[exclude_cols].apply(
                lambda row: ";\n".join([f"{col} : {row[col]}" for col in exclude_cols if pd.notna(row[col])]),
                    axis=1
                )
                # hasil dataframe hanya berisi kolom utama + extra_info
                dt_result = dt_anom[keep_cols + ['extra_info']]
                dt_result = dt_result.sort_values(by=['KODE_KAB','KODE_WILAYAH' ,'DSRT','kode_anomali'])
                unik_kab = dt_result['KODE_KAB'].unique()
                for kab in unik_kab:
                    kab = kab.zfill(2)
                    dt_filter = dt_result[dt_result.KODE_KAB==kab]
                    os.makedirs(f'{folder_output}/{tgl}/{kab}', exist_ok=True)
                    dt_filter.to_excel(f"output/{tgl}/{kab}/{kode_anomali}.xlsx")
                dt_anom_all.append(dt_result)
        except Exception as e:
            print(e)
            print(file)

In [48]:
# Nama file zip
zip_path = f'{folder_input}/{file_anomali_zip_konsistensi}'

os.makedirs(f'{folder_output}/{tgl}', exist_ok=True)
# Buka zip
with zipfile.ZipFile(zip_path, 'r') as z:
    # Lihat daftar file di dalam zip
    # print(z.namelist())

    # Baca salah satu file (misalnya 'data.txt')
    for file in z.namelist():
        try:
            kode_anomali=file.split('_')[1]
            nama_anomali=" ".join(file.split('_')[2:-1])
            with z.open(file) as f:
                dt_anom = pd.read_excel(f,dtype='str')
                dt_anom['kode_anomali']=kode_anomali
                dt_anom['nama_anomali']=nama_anomali
                dt_anom=pd.merge(dt_anom,dt_alokasi,how='left',left_on=['KODE_WILAYAH','DSRT'],right_on=['KODE_WILAYAH','NUS'])
                keep_cols = [
                    'KODE_KAB',
                    'KODE_WILAYAH',
                    'kab_name',
                    'kec_name',
                    'desa_name',
                    'sls_name',
                    'DSRT','KRT','kode_anomali','nama_anomali','catatan','alamat','NO_BANG','longitude','latitude','email','petugas','role','LINK']
                # kolom yang mau digabung (exclude keep_cols)
                exclude_cols = [c for c in dt_anom.columns if c not in keep_cols]

                # buat kolom baru "extra_info"
                dt_anom['extra_info'] = dt_anom[exclude_cols].apply(
                lambda row: ";\n".join([f"{col} : {row[col]}" for col in exclude_cols if pd.notna(row[col])]),
                    axis=1
                )
                # hasil dataframe hanya berisi kolom utama + extra_info
                dt_result = dt_anom[keep_cols + ['extra_info']]
                dt_result = dt_result.sort_values(by=['KODE_KAB','KODE_WILAYAH' ,'DSRT','kode_anomali'])
                unik_kab = dt_result['KODE_KAB'].unique()

                for kab in unik_kab:
                    kab = kab.zfill(2)
                    dt_filter = dt_result[dt_result.KODE_KAB==kab]
                    os.makedirs(f'{folder_output}/{tgl}/{kab}', exist_ok=True)
                    dt_filter.to_excel(f"{folder_output}/{tgl}/{kab}/{kode_anomali}.xlsx")
                dt_anom_all.append(dt_result)
        except Exception as e:
            print(e)
            print(file)

In [49]:
#gabung dan pecah per kab kota
dt_anom_all_df =pd.concat(dt_anom_all)
dt_anom_all_df=dt_anom_all_df.sort_values(by=['KODE_KAB','KODE_WILAYAH' ,'DSRT','kode_anomali'])
unik_kab = dt_anom_all_df['KODE_KAB'].unique()
for kab in unik_kab:
    kab = kab.zfill(2)
    dt_filter = dt_anom_all_df[dt_anom_all_df.KODE_KAB==kab]
    os.makedirs(f'{folder_output}/{tgl}/{kab}', exist_ok=True)
    dt_filter.to_excel(f"{folder_output}/{tgl}/{kab}/anomali_gabung.xlsx")